In [2]:
rm(list = ls())
library(Rcpp)
library(fields)

source("dpfast.R")
source("dist2.R")
source("shapeDTW_translate/samplingSequencesIdx.R")
sourceCpp("shapeDTW_translate/dpcore.cpp")
sourceCpp("shapeDTW_translate/dpcore_window.cpp")

- ``dist2``: computes the pairwise Euclidean distance between two sets of vectors and returns an $n \times m$ matrix of pairwise distances.

- ``samplingSequencesIdx``: samples the given time series at specified indices and returns a cell array of the resulting sub-sequences.

- ``dpfast``: computes the dynamic time warping (DTW) between two time series using a fast implementation

- ``hist_cost_2``: computes the pairwise Chi-square distance between two sets of histograms and returns an $n \times m$ matrix of pairwise distances. 



#### 1. Check ``dist2.R``

In [3]:
set.seed(1)
x <- matrix(rnorm(2*3, mean=2, sd=2), nrow=2)
c <- matrix(runif(5*3), nrow=5)

# Use r built-in distance function to check if the above calculation is correct
print(paste0('Number of different elements from two distance matrices calculated is ', 
      sum((round(dist2(x, c),2) == round(rdist(x,c), 2)) == FALSE)))

[1] "Number of different elements from two distance matrices calculated is 0"


#### 2. Check ``dpfast``

If ``window_size`` is missing, ``dpcore.cpp`` is called, otherwise ``dpcore_window.cpp`` is called.

In [4]:
# Test dpfast and dpcore
nrow = 5
ncol = 4
M = matrix(abs(rnorm(nrow * ncol)), nrow=nrow, ncol=ncol)
C = rbind(c(1, 1, 1), c(1, 0, 1), c(0, 1, 1))
dpfast(M, C, window_size=4)

0.2992151,1.5367535,1.593860,1.640586
0.7107260,0.5234830,1.027091,1.262798
0.9629494,0.9008786,1.609252,1.569979
1.8548705,1.0342150,1.591832,2.003290
2.2905538,1.8384045,2.318814,2.241304


In [5]:
dpfast(M, C, window_size=2) 

0.2992151,1.5367535,1.593860,0.000000
0.7107260,0.5234830,1.027091,1.262798
0.9629494,0.9008786,1.609252,1.569979
0.0000000,1.0342150,1.591832,2.003290
0.0000000,0.0000000,2.318814,2.241304


#### 3. Check ``samplingSequencesIdx``

In [6]:
# Read data from file
data_urc <- read.table('data/dataSet_URC.txt', quote="\"", comment.char="")[,2:8]
samplingSequencesIdx(data_urc, 5, c(3:5))

,V2,V3,V4,V5,V6,V7,V8
1,0.4248307,1.4173710,2.6664697,3.295756,2.2625860,0.1650483,-0.9588735
2,0.6525383,2.1728969,3.6374980,2.274609,0.9763371,-0.3876330,-0.9102164
3,0.4046725,0.5536035,0.7235771,1.447976,2.0111302,1.6232818,1.2038459
4,1.0870582,2.0101909,2.2997473,1.628742,0.6360933,-0.1405643,0.5473488
5,0.4436209,0.9472850,1.9240844,2.159790,1.4994468,0.9645078,0.2232562
,V2,V3,V4,V5,V6,V7,V8
2,0.6525383,2.1728969,3.6374980,2.274609,0.9763371,-0.3876330,-0.9102164
3,0.4046725,0.5536035,0.7235771,1.447976,2.0111302,1.6232818,1.2038459
4,1.0870582,2.0101909,2.2997473,1.628742,0.6360933,-0.1405643,0.5473488
5,0.4436209,0.9472850,1.9240844,2.159790,1.4994468,0.9645078,0.2232562
